---
title: Wind magnetic field data pipeline
---

- We use magnetic field data in **RTN** coordinate system

In [2]:
# | export
import polars as pl

from ids_finder.utils.cdf import cdf2pl
from ids_finder.utils.basic import pmap
from ids_finder.utils.polars import create_partitions
from ids_finder.pipelines.default.data_mag import create_pipeline_template

import pyspedas

from functools import partial

[11/22/23 15:02:31] WARNING  TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook    ]8;id=177693;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=142369;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter                  
                             console)                                                                              
                                                                                                                   

In [3]:
# | hide
# | default_exp pipelines/wind/mag

## Loading data

In [4]:
# | export
def download_data(start: str, end: str, datatype="h4-rtn") -> list[str]:
    trange = [start, end]
    return pyspedas.wind.mfi(trange, datatype=datatype, downloadonly=True)

In [5]:
# | export
def load_data(
    start: str = None,
    end: str = None,
    datatype="h4-rtn",
    var_names="BRTN",
):
    files = download_data(start=start, end=end, datatype=datatype)
    load_func = partial(cdf2pl, var_names=var_names)
    
    return pl.concat(files | pmap(load_func))

## Preprocessing data

In [7]:
# | export
def preprocess_data(
    raw_data,
    var_names="BRTN",
):
    """
    Preprocess the raw dataset (only minor transformations)
    - Applying naming conventions for columns
    """
    
    load_func = partial(cdf2pl, var_names=var_names)
    return create_partitions(raw_data, load_func)

## Pipeline

In [ ]:
#| export
from ids_finder.pipelines.default.data_mag import process_data
from ids_finder import PARAMS

In [1]:
# | export
def create_pipeline(sat_id="Wind", source="MAG", params=None, **kwargs):
    # if params is None:
        # params = PARAMS[sat_id][source]
    return create_pipeline_template(
        sat_id=sat_id,
        source=source,
        # params=params,
        load_data_fn=download_data,
        preprocess_data_fn=preprocess_data,
        process_data_fn=process_data,
        **kwargs,
    )